In [1]:
# pip freeze > requirements.txt

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import configparser
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
import os

2025-11-03 05:46:34.975174: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-03 05:46:35.029773: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-03 05:46:36.571073: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [3]:
config = configparser.ConfigParser()
config.read("config.ini")
OUTPUT_DIR = config["directory"]["result"]
IMG_WIDTH = int(config["image"]["width"])
IMG_HEIGHT = int(config["image"]["height"])
CLASS_WEIGHT = float(config["model"]["class_weight"])
THRESHOLD = float(config["model"]["threshold"])

# Détection Carotid

In [4]:
# Fonction de perte personnalisée compatible avec différentes versions de TensorFlow
def weighted_binary_crossentropy(y_true, y_pred):
    # Poids pour les pixels positifs (carotides)
    pos_weight = CLASS_WEIGHT
    
    # Calculer BCE manuellement pour éviter les problèmes de version
    epsilon = tf.keras.backend.epsilon()
    y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
    bce = -(y_true * tf.math.log(y_pred) + (1.0 - y_true) * tf.math.log(1.0 - y_pred))
    
    # Appliquer les poids
    weighted_bce = bce * (y_true * pos_weight + (1.0 - y_true))
    
    # Retourner la moyenne
    return tf.reduce_mean(weighted_bce)

# Fonction Dice comme métrique
def dice_coefficient(y_true, y_pred, smooth=1.0):
    y_true_f = tf.cast(tf.keras.backend.flatten(y_true), tf.float32)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)
    
def load_trained_model(model_path):
    """
    Charge un modèle pré-entraîné
    
    Args:
        model_path: Chemin vers le fichier du modèle (.h5)
        
    Returns:
        Le modèle chargé
    """
    print(f"Chargement du modèle {model_path}...")
    
    try:
        # Charger le modèle avec les fonctions personnalisées
        model = load_model(model_path, custom_objects={
            'weighted_binary_crossentropy': weighted_binary_crossentropy,
            'dice_coefficient': dice_coefficient
        })
        print("Modèle chargé avec succès!")
        return model
    except Exception as e:
        print(f"Erreur lors du chargement du modèle: {e}")
        
        # Essayer une autre méthode de chargement si la première échoue
        try:
            model = load_model(model_path, compile=False)
            model.compile(
                optimizer=Adam(learning_rate=0.0001),
                loss=weighted_binary_crossentropy,
                metrics=['accuracy', dice_coefficient]
            )
            print("Modèle chargé avec succès (méthode alternative)!")
            return model
        except Exception as e2:
            print(f"Échec de chargement du modèle: {e2}")
            return None

def preprocess_image(image_path):
    """
    Prétraite une image pour la prédiction
    
    Args:
        image_path: Chemin vers l'image à prétraiter
        
    Returns:
        L'image prétraitée, redimensionnée et normalisée
    """
    # Charger l'image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    if image is None:
        print(f"Erreur: Impossible de charger l'image {image_path}")
        return None
    
    # Redimensionner
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    
    # Normaliser
    image = image / 255.0
    
    # Amélioration du contraste (CLAHE)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    image = clahe.apply((image * 255).astype(np.uint8)) / 255.0
    
    return image

# Fonction pour superposer un masque sur une image
def overlay_mask(image, mask, alpha=0.7):
    # Convertir l'image en RGB
    if len(image.shape) == 2 or image.shape[2] == 1:
        image_rgb = cv2.cvtColor((image * 255).astype(np.uint8).reshape(IMG_HEIGHT, IMG_WIDTH), cv2.COLOR_GRAY2RGB)
    else:
        image_rgb = (image * 255).astype(np.uint8)
    
    # Créer une superposition rouge pour les carotides
    overlay = image_rgb.copy()
    binary_mask = (mask > THRESHOLD).astype(np.uint8)
    
    # Assurer que le masque a la bonne forme
    if len(binary_mask.shape) == 3 and binary_mask.shape[2] == 1:
        binary_mask = binary_mask.reshape(IMG_HEIGHT, IMG_WIDTH)
    
    overlay[binary_mask > 0] = [255, 0, 0]  # Rouge
    
    # Mélanger l'image originale et la superposition
    blended = cv2.addWeighted(image_rgb, 1 - alpha, overlay, alpha, 0)
    
    return blended
    
def save_prediction_overlay(image, pred_mask, filename):
    overlay = overlay_mask(image, pred_mask)
    cv2.imwrite(filename, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))
            
def predict_on_image(model, image_path, output_dir):
    """
    Fait une prédiction sur une seule image
    
    Args:
        model: Modèle pré-entraîné
        image_path: Chemin vers l'image à prédire
        output_dir: Répertoire où sauvegarder les résultats
    """
    # Créer le répertoire de sortie s'il n'existe pas
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, "mask"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "overlay"), exist_ok=True)
    
    # Prétraiter l'image
    image = preprocess_image(image_path)
    
    if image is None:
        return
    
    # Préparer l'image pour la prédiction
    input_image = image.reshape(1, IMG_HEIGHT, IMG_WIDTH, 1)
    
    # Faire la prédiction
    prediction = model.predict(input_image)[0]
    
    # Binariser la prédiction
    binary_pred = (prediction > THRESHOLD).astype(np.uint8)
    
    # Extraire le nom de base du fichier
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    
    # Sauvegarder le masque prédit
    mask_path = os.path.join(output_dir, "mask", f"{base_name}_mask.png")
    cv2.imwrite(mask_path, binary_pred.reshape(IMG_HEIGHT, IMG_WIDTH) * 255)
    
    # Sauvegarder l'overlay
    overlay_path = os.path.join(output_dir, "overlay",f"{base_name}_overlay.png")
    save_prediction_overlay(image, binary_pred, overlay_path)
    
    print(f"Prédiction pour {os.path.basename(image_path)} sauvegardée dans {output_dir}")
    
def predict_on_directory(model, input_dir, output_dir):
    """
    Fait des prédictions sur toutes les images d'un répertoire
    
    Args:
        model: Modèle pré-entraîné
        input_dir: Répertoire contenant les images à prédire
        output_dir: Répertoire où sauvegarder les résultats
    """
    # Créer le répertoire de sortie s'il n'existe pas
    os.makedirs(output_dir, exist_ok=True)
    
    # Lister toutes les images du répertoire
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.tif', '*.tiff', '*.bmp']
    image_paths = []
    for ext in image_extensions:
        image_paths.extend(glob(os.path.join(input_dir, ext)))
        image_paths.extend(glob(os.path.join(input_dir, ext.upper())))
    
    if not image_paths:
        print(f"Aucune image trouvée dans {input_dir}")
        return
    
    print(f"Traitement de {len(image_paths)} images...")
    
    # Faire les prédictions sur chaque image
    for i, image_path in enumerate(image_paths):
        print(f"[{i+1}/{len(image_paths)}] Traitement de {os.path.basename(image_path)}...")
        predict_on_image(model, image_path, output_dir)
    
    print(f"Toutes les prédictions ont été sauvegardées dans {output_dir}")

def detection_carotide(model_path, dir):
    """
    Mode test : utilise un modèle pré-entraîné pour faire des prédictions
    
    Args:
        model_path: Chemin vers le fichier du modèle (.h5)
        dir: Répertoire contenant les images
    """
    
    # Vérifier que le modèle existe
    if not os.path.exists(model_path):
        print(f"Erreur: Le modèle {model_path} n'existe pas.")
        return
    
    # Vérifier que le répertoire de test existe
    if not os.path.exists(dir):
        print(f"Erreur: Le répertoire de test {dir} n'existe pas.")
        return
    
    # Charger le modèle
    model = load_trained_model(model_path)
    
    if model is None:
        return
    
    # Créer un répertoire pour les résultats
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # Faire des prédictions sur toutes les images du répertoire de test
    predict_on_directory(model, dir, OUTPUT_DIR)

detection_carotide("carotide_detector_v2.h5", "input")

Chargement du modèle carotide_detector_v2.h5...


2025-11-03 05:46:37.352173: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Erreur lors du chargement du modèle: 'function' object has no attribute 'replace'
Modèle chargé avec succès (méthode alternative)!
Traitement de 28 images...
[1/28] Traitement de carotide22.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step
Prédiction pour carotide22.png sauvegardée dans result
[2/28] Traitement de carotide5.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Prédiction pour carotide5.png sauvegardée dans result
[3/28] Traitement de carotide4.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
Prédiction pour carotide4.png sauvegardée dans result
[4/28] Traitement de carotide23.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
Prédiction pour carotide23.png sauvegardée dans result
[5/28] Traitement de carotide21.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Prédiction pour carotide21.png sauvegardée dans result
[6/28] Traitement de carotide6.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
Prédiction pour carotide6.png sauvegardée dans result
[7/28] Traitement de carotide7.png...
1/1 ━━━━━━━━

# Détection stenose

In [5]:
files = glob(os.path.join(OUTPUT_DIR, "mask", "*.png"))
carotid_left = []
carotid_right = []
for f in files :
    image = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    
    # On transforme en noir/blanc pur : 0 ou 255
    _, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    shapes = []
    
    for i, cnt in enumerate(contours):
        area = cv2.contourArea(cnt)
        M = cv2.moments(cnt)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])  # Coordonnée x du centre
            cy = int(M["m01"] / M["m00"])  # Coordonnée y du centre
            shapes.append({"id": i+1, "area": area, "cx": cx, "cy": cy, "contour": cnt})
    
    shapes.sort(key=lambda s: s["cx"])
    
    left = shapes[0]
    right = shapes[1]
    carotid_left.append(left["area"])
    carotid_right.append(right["area"])

# # 5. Afficher les contours sur l'image originale
# output = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
# cv2.drawContours(output, contours, -1, (0, 0, 255), 2)

# plt.imshow(output[..., ::-1])
# plt.title("Contours détectés")
# plt.show()

### Sténose estimée (moyenne pondérée)

La sténose estimée se calcule par :

$$
\text{Sténose estimée (\%)} = 
\frac{\sum_i w_i \cdot \left( 1 - \frac{A_i}{A_{\max}} \right)}{\sum_i w_i} \times 100
$$

Où :  

- $A_{\max}$ = aire maximale (approximation du diamètre normal)  
- $A_i$ = aire détectée sur l’image $i$  
- $w_i$ = poids de chaque image (par exemple $1$ si toutes les images ont le même poids, ou selon qualité de segmentation)

In [6]:
A_left_max = max(carotid_left)
A_right_max = max(carotid_right)

# Calcul sténose pondérée (poids = 1 ici)
stenosis_left = np.mean([1 - np.sqrt(a / A_left_max) for a in carotid_left]) * 100
stenosis_right = np.mean([1 - np.sqrt(a / A_right_max) for a in carotid_right]) * 100

print(f"Sténose pondérée carotid gauche : {stenosis_left:.2f}%")
print(f"Sténose pondérée carotid droite : {stenosis_right:.2f}%")

Sténose pondérée carotid gauche : 46.47%
Sténose pondérée carotid droite : 64.88%
